# Spatially Informed Traveling Salesman Problem
[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1hDyhN-nOeMh0dJlPpAkndSQZnK_yb82r)    Developed by Wanhee Kim(Phd student at UTK)



## Sequence of spatially informed TSP
1. Measure adjacency based on spatial information
2. Generate distance matrix
3. Analyze TSP

### Install packages

In [ ]:
# basic
import os
import time
import math
import subprocess
from random import sample

# data analysis
import pandas as pd
import numpy as np
import scipy.stats
from sklearn.neighbors import NearestNeighbors

# spatial data
import geopandas as gpd
from shapely.geometry import Point, Polygon, box
from scipy.spatial import Voronoi, voronoi_plot_2d
from scipy.spatial.distance import euclidean
from geopandas.tools import overlay
from descartes import PolygonPatch

# visualize
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from mpl_toolkits.axes_grid1 import make_axes_locatable
import seaborn as sns
import plotly.express as px

# text
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

## Analysis

In [ ]:
# set working directory
path = 'D:/GIS_analyzing/1.Standard_TSP/0.test_iteration' # write your own directory
os.chdir(path)
os.getcwd()

In [ ]:
## Generate Clustered points
# Guarantee a sufficient distance among points
from shapely.geometry import Point

def is_far_enough(new_point, existing_points, min_distance=3):
    for point in existing_points:
        if np.sqrt((new_point[0] - point[0])**2 + (new_point[1] - point[1])**2) < min_distance:
            return False
    return True

# Generate Clustered Points
def generate_clustered_points(num_points, std_dev=5, cluster_centers=[(13, 13), (37, 37)], x_max=50, y_max=50, min_distance=3, seed=None):
    np.random.seed(seed)
    all_points = set()

    points_per_cluster = num_points // len(cluster_centers)
    extra_points = num_points % len(cluster_centers)  #    
    
    for index, center in enumerate(cluster_centers):
        points = set()
        extra = 1 if index < extra_points else 0
        while len(points) < points_per_cluster + extra:
            x = np.random.normal(center[0], std_dev)
            y = np.random.normal(center[1], std_dev)
            x, y = int(round(x)), int(round(y))
            if x > 1 and y > 1 and x <= x_max and y <= y_max and is_far_enough((x, y), all_points, min_distance):
                points.add((x, y))
        all_points.update(points)
    
    points_list = list(all_points)
    gdf_points = gpd.GeoDataFrame({'geometry': [Point(p) for p in points_list]}, crs="EPSG:4326")
    
    return gdf_points

In [ ]:
## Calculate Standard distance
def calculate_distance_matrix(gdf_points):
    points = np.array([[point.x, point.y] for point in gdf_points.geometry])
    num_points = len(points)
    distance_matrix = np.zeros((num_points, num_points), dtype=int)
    for i in range(num_points):
        for j in range(num_points):
            if i != j:
                distance = np.sqrt((points[i][0] - points[j][0])**2 + (points[i][1] - points[j][1])**2)*100
                distance_matrix[i, j] = round(distance)
            else:
                distance_matrix[i, j] = 0
    return distance_matrix

In [ ]:
##  Function to create Voronoi polygons and calculate 1st-order adjacency distance matrix
def voronoi_adjacency_distance(gdf_points, clip_box=box(0, 0, 50, 50)):
    points = np.array([[point.x, point.y] for point in gdf_points.geometry])
    points2=points
    points2 = np.append(points2, [[999,999], [-999,999], [999,-999], [-999,-999]], axis = 0)
    vor = Voronoi(points2)
    point_to_region = vor.point_region[:len(points)]
    polygons = []
    ids = []
    for point_idx, region_idx in enumerate(point_to_region):
        region = vor.regions[region_idx]
        if -1 in region:

            continue
        polygon = Polygon([vor.vertices[i] for i in region])
        clipped_polygon = polygon.intersection(clip_box)
        if not clipped_polygon.is_empty:
            polygons.append(clipped_polygon)
            ids.append(point_idx)  .
    
    voronoi_gdf = gpd.GeoDataFrame({'id': ids, 'geometry': polygons}, crs="EPSG:4326")

    num_points = len(points)
    distances = np.full((num_points, num_points), 99999)
    for i in range(num_points):
        for j in range(num_points):
            if i != j:
                if voronoi_gdf.geometry[i].touches(voronoi_gdf.geometry[j]):
                    distance = gdf_points.geometry[i].distance(gdf_points.geometry[j])*100
                    distances[i, j] = round(distance)    
    return distances, voronoi_gdf

In [ ]:
## other codes will be updated soon